In [2]:
!pip install --quiet -U lancedb
!pip install --quiet gradio transformers torch torchvision


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [14]:
data = tbl.to_lance().to_table()

In [30]:
import lance
import pyarrow.compute as pc

lance.write_dataset(data.filter(~pc.field("prompt").is_null()), "~/datasets/demo/diffusiondb.lance")

In [42]:
import io
import PIL
import duckdb
import lancedb

uri = "~/datasets/demo"
db = lancedb.connect(uri)
tbl = db.open_table("diffusiondb")
tbl.create_fts_index(["prompt"])

In [48]:
from transformers import CLIPModel, CLIPProcessor, CLIPTokenizerFast

MODEL_ID = "openai/clip-vit-base-patch32"

tokenizer = CLIPTokenizerFast.from_pretrained(MODEL_ID)
model = CLIPModel.from_pretrained(MODEL_ID)
processor = CLIPProcessor.from_pretrained(MODEL_ID)

def embed_func(query):
    inputs = tokenizer([query], padding=True, return_tensors="pt")
    text_features = model.get_text_features(**inputs)
    return text_features.detach().numpy()[0]

ninja turtle
turtle
turtle


In [46]:
def find_image_vectors(query):
    emb = embed_func(query)
    return _extract(tbl.search(emb).limit(9).to_df())

def find_image_keywords(query):
    print(query)
    return _extract(tbl.search(query).limit(9).to_df())

def find_image_sql(query):
    diffusiondb = tbl.to_lance()
    return _extract(duckdb.query(query).to_df())

def _extract(df):
    image_col = "image"
    return [(PIL.Image.open(io.BytesIO(row[image_col])), row["prompt"]) for _, row in df.iterrows()]

In [47]:
import gradio as gr


with gr.Blocks() as demo:

    with gr.Row():
        with gr.Tab("Embeddings"):
            vector_query = gr.Textbox(value="geodesic landscape", show_label=False)
            b1 = gr.Button("Submit")
        with gr.Tab("Keywords"):
            keyword_query = gr.Textbox(value="turtle", show_label=False)
            b2 = gr.Button("Submit")
        with gr.Tab("SQL"):
            sql_query = gr.Textbox(value="SELECT * from diffusiondb WHERE image_nsfw >= 2 LIMIT 9", show_label=False)
            b3 = gr.Button("Submit")
    with gr.Row():
        gallery = gr.Gallery(
                label="Found images", show_label=False, elem_id="gallery"
            ).style(columns=[3], rows=[3], object_fit="contain", height="auto")   
        
    b1.click(find_image_vectors, inputs=vector_query, outputs=gallery)
    b2.click(find_image_keywords, inputs=keyword_query, outputs=gallery)
    b3.click(find_image_sql, inputs=sql_query, outputs=gallery)
    
demo.launch()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


ninja turtle
